In [ ]:
""" preparatory block """
import pandas as pd

import common

## 1. Load the collected data into pandas-friendly data structures.

The Excel spreadsheet contains many sheets, each with one participant's data.
For these analyses, all trials will be loaded into a single dataframe.

In [ ]:
def pid_from_sheet(sheet):
    """ Pull characters from sheet name before the first hyphen. """
    return sheet[: sheet.find('-')]

def row_is_data(neutral, hitmiss):
    """ Return True if row contains raw data. """
    if str(hitmiss).lower().strip("\"").strip("'") in ['hit', 'miss']:
        try:
            return 1000 < int(neutral) < 9999
        except TypeError:
            return False
    return False

def row_is_meta(neutral, hitrate):
    """ Return True if the row contains metadata. """
    if str(neutral).lower() in ['session', 'participant', ]:
        return True
    if 'anxiety' in str(hitrate).lower():
        return True
    if 'rate' in str(hitrate).lower():
        return True
    return False

def meta_in_row(row):
    """ Return one metadata (key, value) tuple from one row, with py-friendly naming """
    if 'hit rate' in str(row[6]).lower():
        try:
            return ('xl_hit_rate', float(row[7]))
        except ValueError:
            return ('xl_hit_rate', 0.0)
    elif 'state anxiety' in str(row[6]).lower():
        try:
            return ('state_anxiety', int(row[7]))
        except ValueError:
            return ('state_anxiety', 0)
    elif 'trait anxiety' in str(row[6]).lower():
        try:
            return ('trait_anxiety', int(row[7]))
        except ValueError:
            return ('trait_anxiety', 0)
    elif str(row[1]) in ['session', 'participant', ]:
        return (str(row[1]).lower(), str(row[2]))
    return None

def meta_from_df(df, sheet):
    """ Extract meta-data from the dataframe, return it as a dict. """
    meta_dict = {'sheet': sheet[: -4], 'pid': pid_from_sheet(sheet)}
    if str(df.columns[5]).lower() == 'hit rate':
        # For most sheets, meta 'Hit Rate' is embedded in the top row as a column header.
        # but this check is necessary because exactly one sheet moves it down a row. :(
        meta_dict['xl_hit_rate'] = float(df.columns[6])
    # after harvesting the Hit Rate value, rename it so it can act as an actual column name.
    df = df.rename({df.columns[6]: 'score'}, axis='columns')
    for row in df.itertuples():
        k, v = meta_in_row(row)
        meta_dict[k] = v
    return meta_dict

def harvest_sheets(xlf):
    """ Pull data from each sheet in sheets, return two dataframes. """
    dataframes = []
    metadicts = []
    cols_to_keep = ['Hit_Miss_raw', 'MaskedStim_raw', 'RT_raw', 'choice_raw', 'neutral']
    df_sex = None
    
    xl = pd.ExcelFile(xlf)
    print("{} data sheets were found".format(len(xl.sheet_names)))
    
    for i, sheet in enumerate(xl.sheet_names):
        if sheet.endswith(".tsv"):
            # Each tsv sheet contains complete results for each of 36 trials per participant
            pid = pid_from_sheet(sheet)
            print("  {:03}. loading {} from sheet '{}'".format(i + 1, pid, sheet))
            df = pd.read_excel(xlf, sheet)
            df['sheet'] = sheet[: -4]
            df['pid'] = pid
            df['datetime'] = sheet[sheet.find('-') + 1: -4]
            # The dataframe, df, has everything from the excel sheet, keep the raw data in df_data
            df_data = df[df.apply(lambda x: row_is_data(x[0], x[1]), axis=1)]
            dataframes.append(df_data[['sheet', 'pid', 'datetime', ] + cols_to_keep])
            # The dataframe, df, has everything from the excel sheet, keep the meta-data in df_meta
            df_meta = df[df.apply(lambda x: row_is_meta(x[0], x[5]), axis=1)]
            metadicts.append(meta_from_df(df_meta, sheet))
        elif sheet == "Gender":
            # This sheet appears to be the only place with sex information for all participants
            print("  {:03}. loading sex data from sheet '{}'".format(i + 1, sheet))
            df = pd.read_excel('data.xlsx', 'Gender')
            df_sex_f = df[['Female', ]].reindex()
            df_sex_f.columns = ['pid', ]
            df_sex_f['sex'] = 'F'
            df_sex_m = df[['Male', ]].reindex()
            df_sex_m.columns = ['pid', ]
            df_sex_m['sex'] = 'M'
            df_sex = pd.concat([df_sex_f.dropna(axis=0), df_sex_m.dropna(axis=0), ], axis=0).reindex()
            df_sex['sheet'] = df_sex['pid'].apply(lambda x: x[: -4])
            df_sex = df_sex.set_index('sheet')[['sex', ]]
            # Remove the duplicate sheet name entered twice in the Excel 'Gender' sheet.
            df_sex = df_sex[~df_sex.index.duplicated()]
        else:
            print("       ignoring sheet {}".format(sheet))
    
    df_meta = pd.concat([pd.DataFrame(metadicts).set_index('sheet'), df_sex, ], axis=1, sort=True)
    df_data = pd.concat(dataframes).reindex()
    print("{} data sheets loaded; {} participants found; {:,} trials.".format(
        len(xl.sheet_names), df_meta.shape[0], df_data.shape[0]
    ))
    
    return df_meta, df_data

In [ ]:
""" Read the Excel file and collect most of its data. 
    Save raw-data and meta-data out to separate tsv files.
"""
df_meta, df_data = harvest_sheets(common.source_file)

df_meta.to_csv(common.meta_file, sep="\t")
df_data.to_csv(common.data_file, index_label="sequence", sep="\t")

In [ ]:
# Sanity check
df_meta.sample(8)

In [ ]:
# Sanity check
df_data.sample(8)

In [ ]:
# Error check - We need to drop these from sex comparisons if we do not know the sex.
df_meta[df_meta.isna().any(axis=1)]

In [ ]:
# Error check - We only kept actual data, so probably fine here.
df_data[df_data.isna().any(axis=1)]

In [ ]:
df_sex_f
